In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Load the Iris dataset
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Standardize the data (important for PCA)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.drop('target', axis=1))

# Apply PCA with 2 components
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_scaled)

# Create a DataFrame for visualization
pca_df = pd.DataFrame(df_pca, columns=['PC1', 'PC2'])
pca_df['target'] = df['target']

# Scatter plot of PCA-reduced data
plt.figure(figsize=(8, 6))
colors = ['red', 'green', 'blue']
for target in pca_df['target'].unique():
    subset = pca_df[pca_df['target'] == target]
    plt.scatter(subset['PC1'], subset['PC2'], color=colors[target], label=data.target_names[target])

plt.title('PCA Visualization of Iris Dataset')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()

# Part 2: Stock Price Prediction using ARIMA
# Load stock price dataset
df_stock = pd.read_csv('stock_prices.csv')
df_stock['Date'] = pd.to_datetime(df_stock['Date'])
df_stock.set_index('Date', inplace=True)
df_stock['Close'].fillna(method='ffill', inplace=True)

# Plotting the Close prices
df_stock['Close'].plot(title='Stock Prices Over Time', figsize=(10, 6))
plt.show()

# Train-Test Split
train_size = int(len(df_stock) * 0.8)
train, test = df_stock['Close'][:train_size], df_stock['Close'][train_size:]

# ARIMA Model Training
model = ARIMA(train, order=(5, 1, 0))
model_fit = model.fit()

# Forecasting
forecast = model_fit.forecast(steps=len(test))

# Evaluation
mae = mean_absolute_error(test, forecast)
rmse = np.sqrt(mean_squared_error(test, forecast))

print(f'MAE: {mae:.2f}')
print(f'RMSE: {rmse:.2f}')

# Visualizing Predictions
plt.figure(figsize=(10, 6))
plt.plot(train.index, train, label='Training Data')
plt.plot(test.index, test, label='Actual Prices')
plt.plot(test.index, forecast, label='Forecasted Prices', color='red')
plt.legend()
plt.title('Stock Price Prediction with ARIMA')
plt.show()
